# Mem0 Azure AI Fondry Integration - Quickstart

This guide demonstrates how to use Mem0 with Azure AI Search and Azure OpenAI for adding persistent memory to your AI applications. We'll cover:

- Basic memory operations with Mem0 and Azure AI Search
- A practical travel planning assistant example


In [ ]:
! pip install mem0ai python-dotenv

## Environment Setup
First, set your Azure environment variables:

In [6]:
import os
from mem0 import Memory
from openai import AzureOpenAI

# Load Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

# Load Azure AI Search configuration
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_NAME="fsunavala-ai-search"

# Create Azure OpenAI client
azure_openai_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-10-21"
)

## Basic Memory Operations
Here's a minimal working example of storing and retrieving memories with Mem0 and Azure AI Search:

In [ ]:
# Basic example for Azure AI Search with Mem0
import os
from mem0 import Memory

# Load Azure configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME= os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_NAME = "fsunavala-ai-search"

# Configure Mem0 with Azure AI Search
memory_config = {
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": SEARCH_SERVICE_NAME,
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "memories",
            "embedding_model_dims": 1536,
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            "embedding_dims": 1536,
            "azure_kwargs": {
                "api_version": "2024-10-21",
                "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME, 
                "api_version": "2024-10-21",
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "version": "v1.1",
}
# Initialize memory
memory = Memory.from_config(memory_config)
print("Mem0 initialized with Azure AI Search")

# Basic memory operations
def demonstrate_basic_operations():
    user_id = "demo_user"
    
    # 1. Add a simple memory
    print("\n===== STORING MEMORIES =====")
    memory.add(
        "I enjoy hiking in national parks and taking landscape photos.",
        user_id=user_id
    )
    print("✓ Added memory about hiking preferences")
    
    # 2. Add conversation messages
    conversation = [
        {"role": "user", "content": "I'm planning a trip to Japan in the fall."},
        {"role": "assistant", "content": "That's a great time to visit Japan! The autumn colors are beautiful."},
        {"role": "user", "content": "I'd like to visit Tokyo and Kyoto, and maybe see Mount Fuji."}
    ]
    memory.add(conversation, user_id=user_id)
    print("✓ Added conversation about Japan trip")
    
    # 3. Add memory with metadata
    memory.add(
        "I prefer window seats on long flights and usually bring my own headphones.",
        user_id=user_id,
        metadata={"category": "travel_preferences", "importance": "medium"}
    )
    print("✓ Added travel preferences with metadata")
    
    # 4. Search memories
    print("\n===== SEARCHING MEMORIES =====")
    search_results = memory.search("What are this user's travel plans?", user_id=user_id, limit=3)
    print(f"Found {len(search_results['results'])} relevant memories:")
    for i, result in enumerate(search_results['results'], 1):
        print(f"{i}. {result['memory']} (Score: {result['score']:.4f})")
    
    # 5. Get all memories
    print("\n===== ALL MEMORIES =====")
    all_memories = memory.get_all(user_id=user_id)
    print(f"Total memories: {len(all_memories['results'])}")
    for i, memory_item in enumerate(all_memories['results'], 1):
        print(f"{i}. {memory_item['memory']}")
    
    return "Basic operations demonstrated successfully!"

# Run the demonstration
result = demonstrate_basic_operations()
print(f"\n{result}")

Mem0 initialized with Azure AI Search

===== STORING MEMORIES =====
✓ Added memory about hiking preferences
✓ Added conversation about Japan trip
✓ Added travel preferences with metadata

===== SEARCHING MEMORIES =====
Found 3 relevant memories:
1. Planning a trip to Japan in the fall (Score: 0.6160)
2. Would like to visit Tokyo and Kyoto (Score: 0.6017)
3. Usually brings own headphones on flights (Score: 0.5921)

===== ALL MEMORIES =====
Total memories: 7
1. Maybe see Mount Fuji
2. Usually brings own headphones on flights
3. Enjoy taking landscape photos
4. Planning a trip to Japan in the fall
5. Enjoy hiking in national parks
6. Would like to visit Tokyo and Kyoto
7. Prefers window seats on long flights

Basic operations demonstrated successfully!


## Travel Planning Assistant with Mem0

In [11]:
# Part 2: Travel Planning Assistant with Memory

class TravelAssistant:
    def __init__(self, user_id):
        """Initialize travel assistant with memory for a specific user"""
        self.user_id = user_id
        
        # Configure memory for travel planning
        memory_config = {
            "vector_store": {
                "provider": "azure_ai_search",
                "config": {
                    "service_name": SEARCH_SERVICE_NAME,
                    "api_key": SEARCH_SERVICE_API_KEY,
                    "collection_name": "travel_memories",
                    "embedding_model_dims": 1536,
                    "compression_type": "binary",
                },
            },
            "llm": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                    "temperature": 0.7,
                    "max_tokens": 800,
                    "azure_kwargs": {
                        "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                        "api_version": "2024-10-21",
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "embedder": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    "embedding_dims": 1536,
                    "azure_kwargs": {
                        "api_version": "2024-10-21",
                        "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "version": "v1.1",
        }
        
        self.memory = Memory.from_config(memory_config)
        self.azure_client = azure_openai_client
        print(f"Travel Assistant initialized for user {user_id}")
    
    def get_response(self, query, memory_context=True):
        """Get response from Azure OpenAI with memory context"""
        # Retrieve relevant memories if enabled
        memory_text = ""
        if memory_context:
            memories = self.memory.search(query, user_id=self.user_id)
            if 'results' in memories and memories['results']:
                memory_text = "\n\nRelevant information from previous conversations:\n"
                for i, mem in enumerate(memories['results'][:5], 1):
                    memory_text += f"{i}. {mem['memory']}\n"
                print(f"Including {len(memories['results'][:5])} memories in context")
            else:
                print("No relevant memories found")
        
        # Construct messages with system prompt and memory context
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful travel assistant for London travel planning. "
                           "Be concise, specific, and helpful. Refer to the user by name when appropriate. "
                           "Recommend specific places when relevant."
            },
            {
                "role": "user", 
                "content": f"{query}\n{memory_text}"
            }
        ]
        
        # Get response from Azure OpenAI
        response = self.azure_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            temperature=0.7,
            max_tokens=800,
        )
        
        # Extract response content
        response_content = response.choices[0].message.content
        
        # Store the conversation in memory
        conversation = [
            {"role": "user", "content": query},
            {"role": "assistant", "content": response_content}
        ]
        self.memory.add(conversation, user_id=self.user_id)
        
        return response_content
    
    def verify_memories(self):
        """Verify what memories have been stored"""
        all_memories = self.memory.get_all(user_id=self.user_id)
        print(f"\n===== STORED MEMORIES ({len(all_memories['results'])}) =====")
        for i, memory in enumerate(all_memories['results'], 1):
            print(f"{i}. {memory['memory']}")
        print("==============================\n")
        return all_memories

# Create travel assistant for Farzad
print("\n\n=== TRAVEL PLANNING ASSISTANT DEMO ===\n")
assistant = TravelAssistant(user_id="farzad_london_2025")

# First interaction - Initial inquiry
query1 = "Hi, my name is Farzad. I'm planning a business trip to London next month for about 5 days."
print(f"User: {query1}")
response1 = assistant.get_response(query1, memory_context=False)  # No memories yet
print(f"Assistant: {response1}\n")

# Second interaction - Specific question about fish and chips
query2 = "I need recommendations for fish and chips restaurants near London Bridge cause I love the taste!"
print(f"User: {query2}")
response2 = assistant.get_response(query2)  # Should use memory context
print(f"Assistant: {response2}\n")

# Verify what memories have been stored
assistant.verify_memories()

# Search for Farzad's food preferences
search_query = "What are Farzad's preferences for food in London?"
print(f"Search Query: '{search_query}'")
search_results = assistant.memory.search(search_query, user_id="farzad_london_2025")
print(f"Found {len(search_results['results'])} relevant memories:")
for i, result in enumerate(search_results['results'][:3], 1):
    print(f"{i}. {result['memory']} (Score: {result['score']:.4f})")



=== TRAVEL PLANNING ASSISTANT DEMO ===

Travel Assistant initialized for user farzad_london_2025
User: Hi, my name is Farzad. I'm planning a business trip to London next month for about 5 days.
Assistant: Hi Farzad! That sounds great. For your business trip to London, here are some tips to consider:

1. **Accommodation**: Look for hotels in the City of London or near Canary Wharf for easy access to business hubs. The Montcalm Royal London House or the Hilton London Bankside are good options.

2. **Transportation**: Get an Oyster card or a contactless payment card for convenient travel on the Tube and buses. The London Underground is efficient for getting around the city.

3. **Meeting Venues**: If you need to book meeting rooms, consider venues like the British Library Business & IP Centre or the Barbican Centre, which offer great facilities.

4. **Dining**: For business dinners, try Hawksmoor for steak or Dishoom for a lively Indian dining experience.

5. **Networking**: Look for ne